In [1]:
import pandas as pd
import numpy as np

np.random.seed(42)

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE 

from sklearn.ensemble import GradientBoostingClassifier

from xgboost.sklearn import XGBClassifier
import xgboost as xgb

xgb.set_config(verbosity=0)

In [2]:
data = pd.read_csv('./data/raw_data/data.csv', header=None)
labels = pd.read_csv('./data/raw_data/labels.csv', header=None)

data_labelled = data[:len(labels.index)]
data_unlabelled = data[len(labels.index):]

In [3]:
labels = labels.rename(columns={0: 'target'})
labels['target'] = labels['target'].map({1: False, 2: True})

In [4]:
X = data_labelled.copy()
y = labels.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

## Gradient Boosting Classifier

In [5]:
gradient_boosting_pipeline = Pipeline(
    [
    # ('selector',SelectKBest(f_classif)),
    # ('pca',PCA(random_state=42)),
    ('smote',SMOTE(random_state=42)),
    ('model',GradientBoostingClassifier(random_state=42))
    ]
)

gradient_boosting_search = GridSearchCV(
    estimator = gradient_boosting_pipeline,
    param_grid = {
        # 'selector__k':[5,10,20,50,70,100],
        # 'pca__n_components':[6,9,19],
        'model__learning_rate':[0.1],
        'model__max_depth':[2,3,4,5],
        'model__min_samples_split':[2,3,4],
        'model__min_samples_leaf':[2,5,7,10,20],
        'model__max_features':[3,4,5,6],
        'model__subsample': [0.8],
        'model__n_estimators':[200]
    },
    cv=5,
    n_jobs=-1,
    scoring='f1',
    verbose=1
)
gradient_boosting_search.fit(X_train,y_train.values.ravel())
gradient_boosting_best = gradient_boosting_search.best_estimator_

print('Best parameters:\n', gradient_boosting_search.best_params_)
print('Best score in cross-validation:\n', round(gradient_boosting_search.best_score_,3))
print('Score in test dataset:\n', round(gradient_boosting_search.score(X_test, y_test),3))

y_pred = gradient_boosting_best.predict(X_test)
print('Confusion marix:\n', confusion_matrix(y_true=y_test, y_pred=y_pred))

Fitting 5 folds for each of 240 candidates, totalling 1200 fits
Best parameters:
 {'model__learning_rate': 0.1, 'model__max_depth': 2, 'model__max_features': 4, 'model__min_samples_leaf': 7, 'model__min_samples_split': 2, 'model__n_estimators': 200, 'model__subsample': 0.8}
Best score in cross-validation:
 0.905
Score in test dataset:
 1.0
Confusion marix:
 [[31  0]
 [ 0  5]]


In [6]:
y_pred_validation = gradient_boosting_best.predict(data_unlabelled)
pd.DataFrame(y_pred_validation).value_counts()

False    161
True      19
dtype: int64

## XGBoost

In [8]:
xgboost_pipeline = Pipeline(
    [
    # ('selector',SelectKBest(f_classif)),
    # ('pca',PCA(random_state=42)),
    ('smote',SMOTE(random_state=42)),
    ('model',xgb.XGBClassifier(seed=42, verbosity = 0, silent=True, use_label_encoder =False))
    ]
)

xgboost_search = GridSearchCV(
    estimator = xgboost_pipeline,
    param_grid = {
        'model__eta':[0.1],
        'model__max_depth':[2,3,5],
        'model__min_samples_split':[2,3,4],
        'model__min_samples_leaf':[2,3],
        'model__max_features':[3,4,5,6],
        'model__subsample': [0.8],
        'model__min_child_weight':[1],
        'model__n_estimators':[1000]
    },
    cv=5,
    n_jobs=-1,
    scoring='f1',
    verbose=1
)
xgboost_search.fit(X_train,y_train.values.ravel())
xgboost_best = xgboost_search.best_estimator_

print('Best parameters:\n', xgboost_search.best_params_)
print('Best score in cross-validation:\n', round(xgboost_search.best_score_,3))
print('Score in test dataset:\n', round(xgboost_search.score(X_test, y_test),3))

y_pred = xgboost_best.predict(X_test)
print('Confusion marix:\n', confusion_matrix(y_true=y_test, y_pred=y_pred))

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best parameters:
 {'model__eta': 0.1, 'model__max_depth': 2, 'model__max_features': 3, 'model__min_child_weight': 1, 'model__min_samples_leaf': 2, 'model__min_samples_split': 2, 'model__n_estimators': 1000, 'model__subsample': 0.8}
Best score in cross-validation:
 0.863
Score in test dataset:
 0.909
Confusion marix:
 [[30  1]
 [ 0  5]]
